In [1]:
import spacy
nlp = spacy.load('en')
from spacy import displacy
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [30]:
class Tree:
    def __init__(self, val=None, node=None):
        if val:
            self.val = val
            self.children = []
        if node:
            self.val = node.text
            self.children = []
            for child in node.children:
                self.children.append(Tree(node=child))
                
    def print_level_order(self):
        """
        generate the sequence inorder (include the non-terminals)
        """
        queue = [self, None]
        curr_level = []
        while queue:
            curr = queue.pop(0)
            if curr is None:
                print curr_level
                curr_level = []
                if queue:
                    queue.append(None)
            else:
                curr_level.append(curr.val)
                for child in curr.children:
                    queue.append(child)

In [31]:
with open('data/raw/k3_tree_mini.out', 'r') as f:
    def preprocess(nl):
        return unicode(nl.replace('|', '').replace('.', ''))
    
    def get_root(doc):
        for token in doc:
            if token.dep_ == 'ROOT':
                return token
        raise NoRootFoundException

    lines = [line for line in f]

    for line in lines[:50]:
        nl, fol, pol = line.split('\t')
        nl = preprocess(nl)
        doc = nlp(nl)
        root = Tree(node=get_root(doc))
        # root.print_level_order()
        # displacy.render(doc, jupyter=True, style='dep')

[u'are']
[u'F', u'tetrahedra', u'are']
[u'and', u'Y', u',', u'and', u'X', u'tetrahedra']
[u'and', u'b']
[u'are']
[u'F', u'tetrahedra', u'is']
[u'and', u'y', u'and', u'X', u'tetrahedron', u',', u'and', u'is']
[u'a', u'b', u'tetrahedron']
[u'a']
[u'are']
[u'F', u'tetrahedra', u',', u'and', u'is']
[u'and', u'y', u'x', u'tetrahedron', u',', u'and', u'is']
[u'a', u'b', u'tetrahedron']
[u'a']
[u'are']
[u'F', u'tetrahedra', u',', u'and']
[u'and', u'Y']
[u'are']
[u'F', u'tetrahedra']
[u'and', u'y', u'and']
[u'are']
[u'F', u'tetrahedra', u',', u'and', u'is']
[u'and', u'y', u'X', u'tetrahedron', u'and', u'is']
[u'a', u'b', u'tetrahedron']
[u'a']
[u'are']
[u'F', u'tetrahedra', u',', u'and']
[u'and', u'y']
[u'are']
[u'F', u'tetrahedra', u'and', u'is']
[u'and', u'Y', u'x', u'tetrahedron', u'is']
[u'a', u'and', u'b', u'tetrahedron']
[u'a']
[u'are']
[u'F', u'tetrahedra', u'are']
[u'and', u'y', u'and', u'X', u'and', u'b', u'tetrahedra']
[u'are']
[u'F', u'tetrahedra', u'and', u'is']
[u'and', u'Y', u'x'

In [32]:
displacy.render(doc, jupyter=True, style='dep')

In [4]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

(u'Utah', u'nsubj', u'borders', u'VERB', [])
(u'borders', u'ROOT', u'borders', u'VERB', [Utah, Idaho, .])
(u'Idaho', u'dobj', u'borders', u'VERB', [])
(u'.', u'punct', u'borders', u'VERB', [])


In [5]:
class NoRootFoundException(Exception):
    pass

In [9]:
def conversion(doc):
    def get_root(doc):
        for token in doc:
            if token.dep_ == 'ROOT':
                return token
        raise NoRootFoundException
    
    output = {}
    lookup = {}
    root = get_root(doc)
    output['pred'] = root.text
    lookup[root.text] = 'pred'
    
    print lookup
    
    for child in root.children:
        if child.dep_ == 'nsubj':
#             key = lookup[child.head.text]
            output['nsubj'] = child.text
#             lookup[child.text] = None
        elif child.dep_ == 'dobj':
#             key = lookup[child.head.text]
#             lookup[child.text] = None
            output['dobj'] = child.text
        else:
            key = lookup[child.head.text]
            output[key] = '_'.join([child.text, output[key]])
            lookup[output[key]] = None
    
    print output
    return '%s(%s, %s)' % (output['pred'], output['nsubj'], output['dobj'])

In [10]:
print conversion(doc)
print gold

{u'are': 'pred'}
{'pred': u'are_tetrahedra_are', 'nsubj': u'F'}


KeyError: 'dobj'

In [8]:
doc_sets = [
    u'Alice and Bob are happy.',
    u'Alice is happy, and Bob is happy.',
    u'Alice is happy. Additionally, Bob is happy.',
    u'a is a cube.',
    u'b is a cube.',
    u'a and b are cubes.',
    u'b and a are cubes.'
]

gold_sets = [
    u'cube(a)',
    u'cube(b)',
    u'cube(a) & cube(b)',
    u'cube(b) & cube(a)'
]

In [9]:
for text in doc_sets:
    doc = nlp(text)
    displacy.render(doc, jupyter=True, style='dep')

/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
